In [1]:
# UNI BIG-DATA PROJECT / DQN IMPLEMENTIERUNG
import torch
import torch.nn as nn
import torch.optim as optim
import gym
import random
import math
import time
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

OSError: [WinError 1455] Die Auslagerungsdatei ist zu klein, um diesen Vorgang durchzuführen. Error loading "C:\Users\satte\anaconda3\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

In [ ]:
# if gpu is to be used
use_cuda = torch.cuda.is_available()

device = torch.device("cuda:0" if use_cuda else "cpu")
Tensor = torch.Tensor
LongTensor = torch.LongTensor

env = gym.make('MountainCar-v0')

seed_value = 23
env.seed(seed_value)
torch.manual_seed(seed_value)
random.seed(seed_value)

In [ ]:
###### PARAMS ######
learning_rate = 0.02 # KLEINER MACHEN!!! beim nächsten Test, 0.01 zb
num_episodes = 1000
gamma = 1

hidden_layer = 64

replay_mem_size = 50000
batch_size = 32

egreedy = 0.9
egreedy_final = 0
egreedy_decay = 500

report_interval = 10
score_to_solve = 195

####################
goal_list = []
goal = 0
bars = []

x_val = []
y_val = []
action_list = []

In [ ]:
number_of_inputs = env.observation_space.shape[0]
number_of_outputs = env.action_space.n

def calculate_epsilon(steps_done):
    epsilon = egreedy_final + (egreedy - egreedy_final) * \
              math.exp(-1. * steps_done / egreedy_decay )
    return epsilon

In [ ]:
class ExperienceReplay(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0
 
    def push(self, state, action, new_state, reward, done):
        transition = (state, action, new_state, reward, done)
        
        if self.position >= len(self.memory):
            self.memory.append(transition)
        else:
            self.memory[self.position] = transition
        
        self.position = ( self.position + 1 ) % self.capacity
        
        
    def sample(self, batch_size):
        return zip(*random.sample(self.memory, batch_size))
        
        
    def __len__(self):
        return len(self.memory)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(number_of_inputs,hidden_layer)
        self.linear2 = nn.Linear(hidden_layer,number_of_outputs)

        self.activation = nn.Tanh() # =nn.Sigmoid() AUCH TESTEN!!!!!
        #self.activation = nn.ReLU()
        
        
    def forward(self, x):
        output1 = self.linear1(x)
        output1 = self.activation(output1)
        output2 = self.linear2(output1)

        return output2

In [ ]:
class QNet_Agent(object):
    def __init__(self):
        self.nn = NeuralNetwork().to(device)

        self.loss_func = nn.MSELoss()
        #TODO: try other loss functions
        #self.loss_func = nn.SmoothL1Loss()
        
        #TODO try other optimizers
        self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
        #self.optimizer = optim.RMSprop(params=mynn.parameters(), lr=learning_rate)
        
    def select_action(self,state,epsilon):
        
        random_for_egreedy = torch.rand(1)[0] 
        
        if random_for_egreedy > epsilon:      
            
            with torch.no_grad():
                
                state = Tensor(state).to(device)
                action_from_nn = self.nn(state)
                action = torch.max(action_from_nn,0)[1]
                action = action.item()        
        else:
            action = env.action_space.sample()
        
        return action
    
    def optimize(self):
        
        if (len(memory) < batch_size):
            return
        
        state, action, new_state, reward, done = memory.sample(batch_size)
        
        state = Tensor(state).to(device)
        new_state = Tensor(new_state).to(device)
        reward = Tensor(reward).to(device)
        action = LongTensor(action).to(device)
        done = Tensor(done).to(device)

        new_state_values = self.nn(new_state).detach()
        max_new_state_values = torch.max(new_state_values, 1)[0]
        target_value = reward + ( 1 - done ) * gamma * max_new_state_values
  
        predicted_value = self.nn(state).gather(1, action.unsqueeze(1)).squeeze(1)
        
        loss = self.loss_func(predicted_value, target_value)
    
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        
        #Q[state, action] = reward + gamma * torch.max(Q[new_state])

In [ ]:
memory = ExperienceReplay(replay_mem_size)
qnet_agent = QNet_Agent()

steps_total = []
reward_list = []
ave_reward_list = []

frames_total = 0 
solved_after = 0
solved = False

start_time = time.time()

goal = 0

for i_episode in range(num_episodes):
    
    state = env.reset()
    tot_reward = 0
    reward_list = []
    
    step = 0
    #for step in range(100):
    while True:
        
        step += 1
        frames_total += 1
        
        epsilon = calculate_epsilon(frames_total)
        
        #action = env.action_space.sample()
        action = qnet_agent.select_action(state, epsilon)
        
        new_state, reward, done, info = env.step(action)

        memory.push(state, action, new_state, reward, done)
        qnet_agent.optimize()
        
        state = new_state
        tot_reward += reward
        reward_list.append(tot_reward)
        #env.render()
        
                
        #Visualisierung Actionfarbpalette
        x_val.append(new_state[0])
        y_val.append(new_state[1])
        action_list.append(action)
        
        if done:
            steps_total.append(step)
            ave_reward = np.mean(reward_list)
            ave_reward_list.append(ave_reward)
            
            if new_state[0] >= env.goal_position:
                goal +=1

            if (i_episode+1) % 100 == 0:
                #ave_reward = np.mean(reward_list)
                #ave_reward_list.append(ave_reward)
                reward_list = []
                goal_list.append(goal)
                goal=0
                bars.append(str(i_episode+1))
                
                #Visualisierung
                action_list = pd.Series(action_list)
        
                colors = {0:'blue',1:'lime',2:'red'}
                colors = action_list.apply(lambda x:colors[x])
                labels = ['Left','Right','Nothing']

                fig = plt.figure(figsize=[7,7])
                ax = fig.gca()
                plt.set_cmap('brg')
                surf = ax.scatter(x_val,y_val, s= 10, c=action_list)
                ax.set_xlabel('Position')
                ax.set_ylabel('Velocity')
                ax.set_title('Entwicklung der actions \n der Episoden {} - {}'.format(i_episode+1-100, i_episode+1))
                recs = []
                for i in range(0,3):
                     recs.append(mpatches.Rectangle((0,0),1,1,fc=sorted(colors.unique())[i]))
                plt.legend(recs,labels,loc=4,ncol=3)
                e = i_episode+1

                # WICHTIG: vorher den Ordner "actions" anlegen!!!
                plt.savefig(f"actions_dqn/00{e}_dqn_actions.png", facecolor='w', edgecolor='w') # f"qtable_bilder/{i}.png"
                plt.show()
                x_val = []
                y_val = []
                action_list = []
            
            
            
            break

            env.close()
env.env.close()


In [ ]:
plt.plot((np.arange(len(ave_reward_list)) + 1), ave_reward_list)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward & Episodes')
plt.savefig(f"actions_dqn/ave_reward_dqn_actions.png", facecolor='w', edgecolor='w')
plt.show()

In [ ]:
plt.bar(bars,goal_list, width = 0.8)
plt.xlabel("Episodes")
plt.ylabel("goals")
plt.title("Goals per 100 Episodes")
plt.savefig(f"actions_dqn/goals_per_100_ep_dqn_actions.png", facecolor='w', edgecolor='w')
plt.show

In [ ]:
goal

In [ ]:
goal_list


In [ ]:
len(bars)